LSTM trained on gridded forcings for each station

In [11]:
%load_ext autoreload
%autoreload 2
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from datetime import datetime, timedelta
from sklearn import preprocessing
import netCDF4 as nc
import torch
from torch import nn
from torch.utils.tensorboard import SummaryWriter
from src import load_data, evaluate
import torch.autograd as autograd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
USE_CUDA = False
if torch.cuda.is_available():
    print('CUDA Available')
    USE_CUDA = True
device = torch.device('cuda' if USE_CUDA else 'cpu')

writer = SummaryWriter()

In [3]:
data_runoff = load_data.load_discharge_gr4j_vic()

/home/mgauch/runoff-nn/src/load_data.py:17: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  data = pd.read_csv(os.path.join(dir, f), skiprows=2, skipfooter=1, index_col=False, header=None, names=['runoff'], na_values='-1.2345')
/home/mgauch/miniconda3/envs/gwf/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [4]:
# For each station, read which grid cells belong to its subwatershed
station_cell_mapping = pd.read_csv('data/station_cell_mapping.csv', skiprows=1, names=['station', 'lat', 'lon', 'row', 'col', 'area'])

In [5]:
rdrs_data = load_data.load_rdrs_forcings()

In [6]:
class LSTMRegression(nn.Module):
        def __init__(self, input_dim, hidden_dim, num_layers, batch_size):
            super(LSTMRegression, self).__init__()
            self.batch_size = batch_size
            self.hidden_dim = hidden_dim
            self.num_layers = num_layers
            self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers)
            self.linear = nn.Linear(hidden_dim, 1)
            self.hidden = self.init_hidden()
        def init_hidden(self):
            return (torch.randn(self.num_layers, self.batch_size, self.hidden_dim, device=device),
                    torch.randn(self.num_layers, self.batch_size, self.hidden_dim, device=device))

        def forward(self, input):
            lstm_out, self.hidden = self.lstm(input, self.hidden)
            return self.linear(lstm_out[-1])

In [7]:
predictions = {}
actuals = {}
models = {}
seq_len = 7 * 24
train_start = datetime.strptime('2010-01-01', '%Y-%m-%d') + timedelta(days=seq_len // 24 + 1)
train_end = '2013-12-31'
test_start = '2014-01-01'
test_end = '2014-12-31'

for station in data_runoff['station'].unique():
    print(station)
    station_runoff = data_runoff[data_runoff['station'] == station].set_index('date')
    station_cell_ids = 39 * station_cell_mapping[station_cell_mapping['station'] == station]['col'] \
        + station_cell_mapping[station_cell_mapping['station'] == station]['row']
    station_rdrs = rdrs_data.filter(regex='_(' + '|'.join(map(lambda x: str(x), station_cell_ids)) + ')$', axis=1)
    
    month_onehot = pd.get_dummies(station_rdrs.index.month, prefix='month')
    month_onehot.index = station_rdrs.index
    station_rdrs = station_rdrs.join(month_onehot)
    if any(station_runoff['runoff'].isna()):
        print('Station', station, 'had NA runoff values. Skipping.')
        continue
    
    station_train = station_rdrs.loc[train_start : train_end]
    station_test = station_rdrs.loc[test_start : test_end]
    num_train_days = len(pd.date_range(train_start, train_end, freq='D'))
    
    x = np.zeros((seq_len, len(pd.date_range(train_start, test_end, freq='D')), station_rdrs.shape[1]))
    for day in range(x.shape[1]):
        x[:,day,:] = station_rdrs[train_start - timedelta(hours = seq_len - 1) + timedelta(days=day) : train_start + timedelta(days=day)]
    
    # Scale training data
    scalers = []  # save scalers to apply them to test data later
    x_train = x[:,:num_train_days,:]
    for i in range(x.shape[2]):
        scalers.append(preprocessing.StandardScaler())
        x_train[:,:,i] = scalers[i].fit_transform(x_train[:,:,i].reshape((-1, 1))).reshape(x_train[:,:,i].shape)
    x_train = torch.from_numpy(x_train).float().to(device)
    y_train = torch.from_numpy(station_runoff.loc[train_start:train_end, 'runoff'].to_numpy()).float().to(device)
    
    # Train model
    learning_rate = 2e-3
    patience = 50
    min_improvement = 0.05
    best_loss_model = (-1, np.inf, None)
    
    # Prepare model
    H = 200
    batch_size = 3
    lstm_layers = 2
    model = LSTMRegression(station_rdrs.shape[1], H, lstm_layers, batch_size).to(device)
    loss_fn = torch.nn.MSELoss(reduction='mean')
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    # warm-start if possible
    if best_loss_model[2] is not None:
        model.load_state_dict(best_loss_model[2])
    for epoch in range(300):
        epoch_losses = []
        for i in range(num_train_days // batch_size):
            model.hidden = model.init_hidden()
            y_pred = model(x_train[:,i*batch_size : (i+1)*batch_size,:])

            loss = loss_fn(y_pred, y_train[i*batch_size : (i+1)*batch_size].reshape((batch_size,1))).to(device)
            epoch_losses.append(loss.item())
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        epoch_loss = np.array(epoch_losses).mean()
        print('Epoch', epoch, 'mean loss:', epoch_loss)
        writer.add_scalar('loss_' + station, epoch_loss, epoch)
        if epoch_loss < best_loss_model[1] - min_improvement:
            best_loss_model = (epoch, epoch_loss, model.state_dict())  # new best model
        elif epoch > best_loss_model[0] + patience:
            print('Patience exhausted in epoch {}. Best loss was {}'.format(epoch, best_loss_model[1]))
            break

    print('Using best model from epoch', str(best_loss_model[0]), 'which had loss', str(best_loss_model[1]))
    model.load_state_dict(best_loss_model[2])
    model.eval()        
    
    # scale test data
    x_test = x[:,num_train_days:,:]
    for i in range(x.shape[2]):
        x_test[:,:,i] = scalers[i].transform(x_test[:,:,i].reshape((-1, 1))).reshape(x_test[:,:,i].shape)
    # if batch size doesn't align with number of samples, add dummies to the last batch
    if x_test.shape[1] % batch_size != 0:
        x_test = np.concatenate([x_test, np.zeros((x_test.shape[0], batch_size - (x_test.shape[1] % batch_size), x_test.shape[2]))], axis=1)
    
    x_test = torch.from_numpy(x_test).float().to(device)
    predict = station_runoff[test_start:test_end].copy()
    predict['runoff'] = np.nan
    pred_array = np.array([])
    print('  Predicting')
    for i in range(x_test.shape[1] // batch_size):
        pred_array = np.concatenate([pred_array, model(x_test[:,i*batch_size : (i+1)*batch_size,:]).detach().cpu().numpy().reshape(batch_size)])
    predict['runoff'] = pred_array[:predict.shape[0]]  # ignore dummies
    predictions[station] = predict
    actuals[station] = station_runoff['runoff'].loc[test_start:test_end]
    models[station] = model

04159900
0 0 0.40845727920532227
0 50 20.37786865234375
0 100 2.7192904949188232
0 150 50.11459732055664
0 200 10.356249809265137
0 250 13.40312671661377
0 300 0.2229377031326294
0 350 0.0032487232238054276
0 400 81.3934555053711
0 450 0.06006290018558502
Epoch 0 mean loss: 38.227202898923444
1 0 0.028435004875063896
1 50 4.141575336456299
1 100 0.30149784684181213
1 150 112.04092407226562
1 200 4.76129674911499
1 250 6.335275650024414
1 300 1.3902827501296997
1 350 0.16071422398090363
1 400 88.43710327148438
1 450 0.6751074194908142
Epoch 1 mean loss: 38.539039903043246
2 0 0.0006835105014033616
2 50 2.410907745361328
2 100 0.18490473926067352
2 150 136.69552612304688
2 200 1.4280966520309448
2 250 6.8669819831848145
2 300 0.824349582195282
2 350 0.5343207716941833
2 400 91.5462646484375
2 450 0.31376025080680847
Epoch 2 mean loss: 38.626411865858906
3 0 0.2060178518295288
3 50 2.441100835800171
3 100 0.4159534275531769
3 150 85.53301239013672
3 200 2.1696078777313232
3 250 4.65518712

In [8]:
nse_list = []
for station, predict in predictions.items():
    nse = evaluate.evaluate_daily(station, predict['runoff'], actuals[station], writer=writer)
    nse_list.append(nse)
    
    print(station, '\tNSE: (clipped to 0)', nse_list[-1])

print('Median NSE (clipped to 0)', np.median(nse_list), '/ Min', np.min(nse_list), '/ Max', np.max(nse_list))

/home/mgauch/runoff-nn/gwf/lib/python3.6/site-packages/pandas/plotting/_converter.py:129: FutureWarning: Using an implicitly registered datetime converter for a matplotlib plotting method. The converter was registered by pandas on import. Future versions of pandas will require you to explicitly register matplotlib converters.

To register the converters:
	>>> from pandas.plotting import register_matplotlib_converters
	>>> register_matplotlib_converters()
  warnings.warn(msg, FutureWarning)


04159900 	NSE: (clipped to 0) -0.23434024997243696
02GC010 	NSE: (clipped to 0) -0.08783950751069813
04215500 	NSE: (clipped to 0) 0.08979987052586269
04174500 	NSE: (clipped to 0) -0.09198994059752885
04165500 	NSE: (clipped to 0) 0.2663229297506492
02GB001 	NSE: (clipped to 0) 0.08386515638513692
04200500 	NSE: (clipped to 0) 0.17346421438740922
04199500 	NSE: (clipped to 0) 0.09909385787701519
04177000 	NSE: (clipped to 0) 0.03856667829648408
04208504 	NSE: (clipped to 0) 0.14052483479513356
04207200 	NSE: (clipped to 0) 0.246536523869465
04213000 	NSE: (clipped to 0) 0.06067604061856269
02GE007 	NSE: (clipped to 0) 0.014310706852164068
02GG009 	NSE: (clipped to 0) -0.12698172568432553
04195820 	NSE: (clipped to 0) -0.20624386079199852
04159492 	NSE: (clipped to 0) -0.25964905224714263
04161820 	NSE: (clipped to 0) 0.014748483021072345
02GG003 	NSE: (clipped to 0) 0.32526291106033955
04196800 	NSE: (clipped to 0) -0.08121012119230997
04215000 	NSE: (clipped to 0) 0.291357391698707
0

In [9]:
writer.close()

In [10]:
load_data.pickle_results('LSTM_VIC', (predictions, actuals), models=models)